# Initializing the SimplicityHL Python Interface

Additional helper functions are located in the `lib` subfolder of this project.  
These helper functions simplify the process of compiling smart contracts.


In [1]:
import pysimplicityhl
import pyhalsimplicity
import json
import sys
sys.path.append("..")
import lib


In [2]:
from pathlib import Path

_temp_folder = Path("../_tmp")
_temp_folder.mkdir(parents=True, exist_ok=True)  # creates folder if not existing
lib.clear_folder(_temp_folder)

#filename, file_ptr = lib.create_temp_file(directory = "../_tmp", suffix = ".txt", delete=false)


# Introdcution

### Simple Contract

In [3]:
contract_code = """
fn main() {
}
"""
contract_witness = """
{}
"""

data = lib.compile(contract_code, contract_witness, folder=_temp_folder)
print(data)

{'status': 'success', 'program': 'JA==', 'witness': ''}


In [4]:
contract_code = """

fn checksig(pk: Pubkey, sig: Signature) {
    let msg: u256 = jet::sig_all_hash();
    jet::bip_0340_verify((pk, msg), sig);
}

fn recursive_covenant() {
    assert!(jet::eq_32(jet::num_outputs(), 2));
    let this_script_hash: u256 = jet::current_script_hash();
    let output_script_hash: u256 = unwrap(jet::output_script_hash(0));
    assert!(jet::eq_256(this_script_hash, output_script_hash));
    assert!(unwrap(jet::output_is_fee(1)));
}

fn inherit_spend(inheritor_pk: Pubkey, inheritor_sig: Signature) {
    let days_180: Distance = 25920;
    jet::check_lock_distance(days_180);
    checksig(inheritor_pk, inheritor_sig);
}

fn cold_spend(cold_pk: Pubkey, cold_sig: Signature) {
    checksig(cold_pk, cold_sig);
}

fn refresh_spend(hot_pk: Pubkey, hot_sig: Signature) {
    checksig(hot_pk, hot_sig);
    recursive_covenant();
}

fn main() {
    let alice_pk: Pubkey = witness::ALICE_PUBLIC_KEY;
    let bob_pk: Pubkey = witness::BOB_PUBLIC_KEY;
    let charlie_pk: Pubkey = witness::CHARLIE_PUBLIC_KEY;
    
    match witness::INHERIT_OR_NOT {
        Left(inheritor_sig: Signature) => inherit_spend(alice_pk, inheritor_sig),
        Right(cold_or_hot: Either<Signature, Signature>) => match cold_or_hot {
            Left(cold_sig: Signature) => cold_spend(bob_pk, cold_sig),
            Right(hot_sig: Signature) => refresh_spend(charlie_pk, hot_sig),
        },
    }
}
"""
contract_witness = """
{
    "INHERIT_OR_NOT": {
        "value": "Left(0x755201bb62b0a8b8d18fd12fc02951ea3998ba42bfc6664daaf8a0d2298cad43cdc21358c7c82f37654275dc2fea8c858adbe97bac92828b498a5a237004db6f)",
        "type": "Either<Signature, Either<Signature, Signature>>"
    },
    "ALICE_PUBLIC_KEY": {
        "value": "0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798",
        "type": "u256"
    },
    "BOB_PUBLIC_KEY": {
        "value": "0xc6047f9441ed7d6d3045406e95c07cd85c778e4b8cef3ca7abac09b95c709ee5",
        "type": "u256"
    },
    "CHARLIE_PUBLIC_KEY": {
        "value": "0xf9308a019258c31049344f85f89d5229b531c845836f99b08601f113bce036f9",
        "type": "u256"
    }

}
"""

data = lib.compile(contract_code, contract_witness, folder=_temp_folder, delete_temp_files=False)
print(data)
locals().update(data)
#print(status)
#print(program)
#print(witness)
#print(code_file)
#print(witness_file)


{'status': 'success', 'program': '5yHQKHQKHQKHQKEGHHHIMFSbFlQAQgUJIQYKvKcwYulQXr0Fu/BAWgC5j7LWeBAl+pI9pt9ZFn0rcXaN1FhgGI3Bzj8EOFROCCcF4cziqISK6EzABFXCwYvClhPYGFQusJfripGQssOAVt34AhgGIThCSD7zcTig/ABQLJsYc113LwehOp6MgcZy/xtyV5RlPnsqZbVWnstdoxRHioxDhgGITafj0EA4CBwoJxSBiLiIBmDi4DjQgxAoNyAOOONx+KBcX8WgcDE5AAcHBiNyLOOE1JIEnwxOZ+Cq1myCZY7Ax/Klosw4w/YaiSrNDaOhM/+erWaIc9xgwDFZAAAAAQNIVC1M24nMERf6gONImmAwub35L6eiVhI0f2X8hANtDAEy06t3vxQYBiFwBg5gXe+1Cj7rQ+ji6QJDRW0vEmFLGZXcRhKAN6Nt0/l2ADDAMXw4Ic3idDCRwDPfEbxc9Hgqw6MFFnfB5SG98/3X6ptShJKOWBgHCQnLZJFkAAAAAAoVfDUc2gh0FinqrkqEEWyo/WTDVDT/U2bJS9gAmPKXAmHgCyTBgGIWkhIoZ0QznIWef/b40z+a+nNzThyQhoT+7cjE0KYRLTvzYTFwuXQKObvJWcC0jz5LzKXn0/Nn/OQnPezJTiq+w46I+xAB5sdEwwDUE5iUkC5guQgozboObaVZ2p0jgwi+XQBxtGGWFy+/xoSyiAw9mWEcVuKG05XwwDELQb6EE/c3ifyUoNc1ZzkPnUg6gZ3G/jPcz7k+eRVKFshBw8VgVZAAAAAINwW58NhzSoNzoMc1Ae/7Z55CGHj0gOG/ZVBjb5kbDqY2kAJh07WGAYhcEISKEGB4jAwHMDLqyIAl/t8mPC4rRGEfM1lVH7CAxNfptKDrOKJGwAHAYBrC4gNL6lcZ9HTU3CRlRyCoGVZzuEYuSi/jp604WRZrD9L3pYeQYFCcSgcthcLAcu

### Debug the results


#### showing info

In [5]:
parameter = ["simplicity", "info", program]
parameter_txt = " ".join(parameter)
data2=json.loads(pyhalsimplicity.run_cli_command(parameter_txt))

#locals().update(data2)
#print(commit_decode)
#print(lib.pretty_print_code(commit_decode))
print(data2)
#print(contract_code)


{'jets': 'core', 'commit_base64': '5yHQKHQKHQKHQKEGHHHIMFSbFlQAQgUJIQYKvKcwYulQXr0Fu/BAWgC5j7LWeBAl+pI9pt9ZFn0rcXaN1FhgGI3Bzj8EOFROCCcF4cziqISK6EzABFXCwYvClhPYGFQusJfripGQssOAVt34AhgGIThCSD7zcTig/ABQLJsYc113LwehOp6MgcZy/xtyV5RlPnsqZbVWnstdoxRHioxDhgGITafj0EA4CBwoJxSBiLiIBmDi4DjQgxAoNyAOOONx+KBcX8WgcDE5AAcHBiNyLOOE1JIEnwxOZ+Cq1myCZY7Ax/Klosw4w/YaiSrNDaOhM/+erWaIc9xgwDFZAAAAAQNIVC1M24nMERf6gONImmAwub35L6eiVhI0f2X8hANtDAEy06t3vxQYBiFwBg5gXe+1Cj7rQ+ji6QJDRW0vEmFLGZXcRhKAN6Nt0/l2ADDAMXw4Ic3idDCRwDPfEbxc9Hgqw6MFFnfB5SG98/3X6ptShJKOWBgHCQnLZJFkAAAAAAoVfDUc2gh0FinqrkqEEWyo/WTDVDT/U2bJS9gAmPKXAmHgCyTBgGIWkhIoZ0QznIWef/b40z+a+nNzThyQhoT+7cjE0KYRLTvzYTFwuXQKObvJWcC0jz5LzKXn0/Nn/OQnPezJTiq+w46I+xAB5sdEwwDUE5iUkC5guQgozboObaVZ2p0jgwi+XQBxtGGWFy+/xoSyiAw9mWEcVuKG05XwwDELQb6EE/c3ifyUoNc1ZzkPnUg6gZ3G/jPcz7k+eRVKFshBw8VgVZAAAAAINwW58NhzSoNzoMc1Ae/7Z55CGHj0gOG/ZVBjb5kbDqY2kAJh07WGAYhcEISKEGB4jAwHMDLqyIAl/t8mPC4rRGEfM1lVH7CAxNfptKDrOKJGwAHAYBrC4gNL6lcZ9HTU3CRlRyCoGVZzuEYuSi/jp604WRZrD9L3pYeQYFCcSgcthcLAc

In [6]:
_sig_hash="020000000101b33e0e092f2f229bb472f7ac15b22783908ebf5b70d9fefa13fc627979b2ca6c0000000000000000000201499a818545f6bae39fc03b637f2a4e1e64e590cac1bc3a6f6d71aa4443654c140100000000000182b800160014b58c22151f4ba159e2255767472ac89137e8183001499a818545f6bae39fc03b637f2a4e1e64e590cac1bc3a6f6d71aa4443654c140100000000000003e8000000000000000004609bef8d556d80e43ae7e0becb3a7e6838b95defe45896ed6075bb9035d06c9964e09e91b2ee81dd61d97ec6e83bfdb03c28f79e0e7038a98964ea5c29cde0b2319878a86dc9e5a0d00269215a43754755a6e173246ad7d330eb82d27e779fd90afd7201e4fba0509b4df120e1d320451f14172c46476646daf8d0d6da80e84c986cc5e073f80ed4dcf0210284187248126ac8e671544245742660022ae160c5e14b09ec0c2a17584bf5c548c85961c02b6efc010c03109ad2420c3f00140b16ab91cd75dcbc1e84ea7a320719cbfc6dc95e5194f9eca996d55a7b2d768c511e2a310e1806240a1241b70a35627302ef7da851f75a1f471748121a2b6978930a58ccaee2309401bd1b6e9fcbb0018601881a80e12071190284906e2a37159c2a162cdba0e67e0aad66c82658ec0c7f2a5a2cc38c3f61a892acd0da3a133ff9ead668873dc60c0310b5b0730445fea038d226980c2e6f7e4be9e895848d1fd97f2100db43004cb4eaddefc50601885c078170e6f13a1848e019ef88de2e7a3c1561d1828b3be0f290def9feebf54da94249472c0c0312050920fc8238dc861438a059b630e6ef256702d23cf92f32979f4fcd9ff3909cf7b32538aafb0e3a23ec40079b1d130c03103785c207e4c8201c5a072580e4e0207fd424f70498ef2fb6dd05ffbb7368dc796e6c47f24404e0b1ff138cfce89a7a21bff5919fa64ce45f8306849072b26c1bfdd2937e6b81774796ff372bd1eb5362d20000000000 0 7fd424f70498ef2fb6dd05ffbb7368dc796e6c47f24404e0b1ff138cfce89a7a bff5919fa64ce45f8306849072b26c1bfdd2937e6b81774796ff372bd1eb5362d2 -i '51202718ee832dc8e696e3d8d013e1ebd99c602ec3507c1b940811919bcbcabbbff3:144c654344aa716d6f3abcc1ca90e5641e4e2a7f633bc09fe3baf64585819a49:0.001' -p 9bef8d556d80e43ae7e0becb3a7e6838b95defe45896ed6075bb9035d06c9964 -s e09e91b2ee81dd61d97ec6e83bfdb03c28f79e0e7038a98964ea5c29cde0b2319878a86dc9e5a0d00269215a43754755a6e173246ad7d330eb82d27e779fd90a"

parameter = ["simplicity", "sighash", _sig_hash]
parameter_txt = " ".join(parameter)
data3=json.loads(pyhalsimplicity.run_cli_command(parameter_txt))

print(data3)

{'sighash': 'd832133eba9525e9e452752e6b2193b3c71084af75832b385a4c108f8100947d', 'signature': None, 'valid_signature': True}


In [7]:
from bitcoinrpc.authproxy import AuthServiceProxy, JSONRPCException

def get_raw_transaction(txid: str, rpc_user: str, rpc_password: str, host="127.0.0.1", port=18332):
    # Verbindung zur Bitcoin Core RPC-Schnittstelle (Testnet)
    rpc_url = f"http://{rpc_user}:{rpc_password}@{host}:{port}"
    rpc_connection = AuthServiceProxy(rpc_url)
    try:
        raw_tx = rpc_connection.getrawtransaction(txid)
        return raw_tx
    except JSONRPCException as e:
        return f"Error: {e}"

# Beispiel Testnet txid (Dummywert, muss gültige Testnet-Transaktion sein)
test_txid = "f5d8ee39a430901c91b0d482f1c4baef6ba6dbb3f994f6e67e6a001d9a1d5bba"
rpc_user = "dein_rpc_user"
rpc_password = "dein_rpc_password"
#raw_transaction = get_raw_transaction(test_txid, rpc_user, rpc_password)
#print(f"Raw Transaction Hex:\n{raw_transaction}")
